# Assigment 3

Reproduce the analysis of the Hard to Employ programs for NY regarding the proportion of 'Ever employed in a CEO transitional job' for former prisoners participating in the program in comparison with the ones who didn't. 

In [35]:
import pandas as pd
import numpy as np
import os
import sys
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline

## Hypothesis test

### Null Hypothesis: the % of former prisoners employed in a CEO transitional job after release is the same or lower for candidates who participated in the program ($P_p$) as for the control group ($P_c$)

* $H_0: P_p <= P_c $    
    
* $H_a: P_p > P_c $
    
## Confidence level

* $\alpha$ = 0.05    



In [36]:
alpha=0.05

#proportions
pc = 3.5*0.01 
pp = 70.1*0.01

#sample sizes
n_c = 409
n_p = 564

#counts
Nt_p = pp * n_p
Nt_c = pc * n_c

In [37]:
sp = (pc * n_c + pp * n_p) / (n_c + n_p)
print (sp)

0.421047276465


In [38]:
def stdev(p,n1,n2):
    return np.sqrt( p * ( 1 - p ) / n1 +  p * ( 1 - p ) / n2)

s = stdev(sp,n_c,n_p)

def zscore(x,mu,s):
    return (x - mu) / s

Zvalue = zscore(x = pp, mu = pc, s = s)
print zPoint

20.7697865408


In [39]:
probZvalue = st.norm.cdf(1 - Zvalue)
print probZvalue

2.71024051708e-87


In [40]:
def report_result(p,a):
    print ('is the p value {0:.2f} smaller than the critical value {1:.2f}? '.format(p,a))
    if p < a:
        print ("YES!")
    else: 
        print ("NO!")
    
    print ('the Null hypothesis is {}'.format( 'rejected' if p < a  else 'not rejected') )

    
report_result(probZvalue,alpha)

is the p value 0.00 smaller than the critical value 0.05? 
YES!
the Null hypothesis is rejected


## Test for impact of the program in Recidivism (Years 1-3)

Does the program has an impact in the conviction of felonies within the first 3 years of release?


## Hypothesis test

### Null Hypothesis: the % of former prisoners convicted of a felony within the third year of their release is the same or lower for candidates who didn't participated in the program ($P_c$) as for the ones that did ($P_p$)

* $H_0: P_{felon_c} <= P_{felon_p} $    
    
* $H_a: P_{felon_c} > P_{felon_p}$
    
## Confidence level

* $\alpha$ = 0.05    

In [42]:
#Proportions
pfc = 11.7 * 0.01  
pfp = 10.0 * 0.01 


#sample sizes remain the same

#counts
Ntf_p = pfp * n_p
Ntf_c = pfc * n_c

#sample sizes
nf_c = 409
nf_p = 568

spf = (pfc * nf_c + pfp * nf_p) / (nf_c + nf_p)

sf = stdev(spf,nf_c,nf_p)

Zfvalue = zscore(x = pfp, mu = pfc, s = sf)

probZfvalue = st.norm.cdf(Zfvalue)

report_result(probZfvalue,alpha)


is the p value 0.20 smaller than the critical value 0.05? 
NO!
the Null hypothesis is not rejected


## Chi squared test ($\chi^2$)

### Employment

In [59]:
observedTable = pd.DataFrame({'employed':[Nt_p,Nt_c],'unemployed':[(1 - pp) * n_p,(1 - pc) * n_c]},index = ['program','control'])

In [60]:
observedTable

,employed,unemployed
program,395.364,168.636
control,14.315,394.685


In [63]:
colSum = observedTable.sum()
print colSum
rowSum = observedTable.sum(1)
print rowSum

employed      409.679
unemployed    563.321
dtype: float64
program    564.0
control    409.0
dtype: float64


In [61]:
N = n_c + n_p
N

973

In [65]:
expectedTable = pd.DataFrame({'employed':[(rowSum[0]*colSum[0])/N,(rowSum[1]*colSum[0])/N],
                              'unemployed':[(rowSum[0]*colSum[1])/N,(rowSum[1]*colSum[1])/N]},
                             index = ['program','control'])

In [66]:
expectedTable

,employed,unemployed
program,237.470664,326.529336
control,172.208336,236.791664


In [94]:
chi2value = 0
for i in range(2):
    for j in range(2):
        o = observedTable.iloc[i,j]
        e = expectedTable.iloc[i,j]
        chi = ((o - e) ** 2) / e
        chi2value = chi2value + chi
print chi2value

431.384032951


In [95]:
df = 1


In [99]:
def chi2test(chi2value,alpha,df):
    chi2Critic = st.chi2.isf(alpha, df = df)
    print 'Is the chi2 value obtained (%1.2f) bigger than the theoretical chi2 (%1.2f) for the assumed confidence level (%1.2f) and degrees of freedom (df = %d)?' %(chi2value,chi2Critic,alpha,df) 
    if chi2value > chi2Critic:
        print ("YES!")
    else: 
        print ("NO!")
    
    print ('the Null hypothesis is {}'.format( 'rejected' if chi2value > chi2Critic else 'not rejected') )

    
chi2test(chi2value,alpha,df)

Is the chi2 value obtained (431.38) bigger than the theoretical chi2 (3.84) for the assumed confidence level (0.05) and degrees of freedom (df = 1)?
YES!
the Null hypothesis is rejected


### Recidivism

In [105]:
observedTablef = pd.DataFrame({'convicted':[Ntf_p,Ntf_c],
                              'notconvicted':[(1 - pfp) * nf_p,(1 - pfc) * nf_c]},
                             index = ['program','control'])
observedTablef

,convicted,notconvicted
program,56.400,511.200
control,47.853,361.147


In [106]:
colSumf = observedTablef.sum()
print colSumf
rowSumf = observedTablef.sum(1)
print rowSumf

convicted       104.253
notconvicted    872.347
dtype: float64
program    567.6
control    409.0
dtype: float64


In [107]:
Nf = nf_c + nf_p
Nf

977

In [108]:
expectedTablef = pd.DataFrame({'convicted':[(rowSumf[0]*colSumf[0])/Nf,(rowSumf[1]*colSumf[0])/Nf],
                              'notconvicted':[(rowSumf[0]*colSumf[1])/Nf,(rowSumf[1]*colSumf[1])/Nf]},
                             index = ['program','control'])
expectedTablef

,convicted,notconvicted
program,60.567045,506.800570
control,43.643272,365.189276


In [111]:
chi2valuef = 0
for i in range(2):
    for j in range(2):
        o = observedTablef.iloc[i,j]
        e = expectedTablef.iloc[i,j]
        chi = ((o - e) ** 2) / e
        chi2valuef = chi2valuef + chi
print chi2valuef

0.77568980952


In [112]:
df = 1

In [113]:
def chi2test(chi2value,alpha,df):
    chi2Critic = st.chi2.isf(alpha, df = df)
    print 'Is the chi2 value obtained (%1.2f) bigger than the theoretical chi2 (%1.2f) for the assumed confidence level (%1.2f) and degrees of freedom (df = %d)?' %(chi2value,chi2Critic,alpha,df) 
    if chi2value > chi2Critic:
        print ("YES!")
    else: 
        print ("NO!")
    
    print ('the Null hypothesis is {}'.format( 'rejected' if chi2value > chi2Critic else 'not rejected') )

    
chi2test(chi2valuef,alpha,df)

Is the chi2 value obtained (0.78) bigger than the theoretical chi2 (3.84) for the assumed confidence level (0.05) and degrees of freedom (df = 1)?
NO!
the Null hypothesis is not rejected
